In [1]:
import model_based_mvpa as mbmvpa
from model_based_mvpa.preprocessing.bids import *
from model_based_mvpa.preprocessing.events import *
from model_based_mvpa.utils.func import *
from model_based_mvpa.models.model import *

In [2]:
root = '/data2/project_model_based_fmri/ds001882/'

In [3]:
def prep_func(row,info):

    new_df = row.copy()
    new_df['subjID'] = info['subject']
    new_df['run'] = info['run']
    if row['delay_left'] >= row['delay_right']:
        new_df['delay_later'] = row['delay_left']
        new_df['delay_sooner'] = row['delay_right']
        new_df['amount_later'] = row['money_left']
        new_df['amount_sooner'] = row['money_right']
        new_df['choice'] = 1 if row['choice'] == 1 else 0
    else:
        new_df['delay_later'] = row['delay_right']
        new_df['delay_sooner'] = row['delay_left']
        new_df['amount_later'] = row['money_right']
        new_df['amount_sooner'] = row['money_left']
        new_df['choice'] = 1 if row['choice'] == 2 else 0
        
    return new_df

In [4]:
def cond_func(df_row):
    return df_row['agent']==0

In [5]:
def latent_func(row,info,k,beta):
    
    new_df = row.copy()
    new_df['subjID'] = info['subject']
    new_df['run'] = info['run']
    
    if row['delay_left'] >= row['delay_right']:
        ev_later   = row['money_left'] / (1 + k * row['delay_left'])
        ev_sooner  = row['money_right'] / (1 + k * row['delay_right'])
        utility = ev_later - ev_sooner
    else:
        ev_sooner   = row['money_left'] / (1 + k * row['delay_left'])
        ev_later  = row['money_right'] / (1 + k * row['delay_right'])
        utility = ev_later - ev_sooner
    new_df['modulation'] = utility
    
    return new_df

In [6]:
par_names = ['k','beta']

In [7]:
dm_model = 'dd_hyperbolic'

In [8]:
save_path = '.'

In [9]:
dm_model, df_events, signals, time_masks = preprocess_events(root, dm_model,   
                                                              latent_func,par_names,
                                                              prep_func,
                                                              cond_func,
                                                              df_events=None,
                                                              all_ind_pars=None,
                                                              use_duration=False,
                                                              layout=None,
                                                              hrf_model='glover',
                                                              save_path=save_path,
                                                              save=True,
                                                              ncore=4,
                                                              time_check=True)


'\ndm_model, df_events, signals, time_masks = preprocess_events(root, dm_model,   \n                                                              latent_func,par_names,\n                                                              prep_func,\n                                                              cond_func,\n                                                              df_events=None,\n                                                              all_ind_pars=None,\n                                                              use_duration=False,\n                                                              layout=None,\n                                                              hrf_model=\'glover\',\n                                                              save_path=save_path,\n                                                              save=True,\n                                                              ncore=os.cpu_count(),\n                                  